In [ ]:
from Acquire.Client import User, Authorisation, Drive, Location, Cheque, \
                           Account, Service
from Acquire.Access import RunRequest

In [ ]:
base_url = "https://fn.acquire-aaai.com/t"
identity_url = "%s/identity" % base_url
storage_url = "%s/storage" % base_url
access_url = "%s/access" % base_url
accounting_url = "%s/accounting" % base_url

In [ ]:
username = input("Please enter your username: ")

In [ ]:
user = User(username=username, service=identity_url)

In [ ]:
result = user.request_login()
from Acquire.Client import create_qrcode
create_qrcode(result["login_url"])

In [ ]:
user.wait_for_login()

In [ ]:
creds = StorageCreds(user=user, service_url=storage_url)
drive = Drive("testjob", creds=creds)

In [ ]:
files = drive.list_files("example_sim")

if len(files) == 0:
    drive.upload("example_sim")
    files = drive.list_files("example_sim")

In [ ]:
location = Location(drive_guid=drive.metadata().guid(), filename="example_sim")
print(location)

In [ ]:
r = RunRequest(image="docker://test_image", input=location)
print(r)

In [ ]:
account = Account(user=user, account_name="main", accounting_url=accounting_url)
print(account)

In [ ]:
deposit(user=user, value=10.0, description="depositing funds", account_name="main",
        accounting_url=accounting_url)

In [ ]:
account.balance()

In [ ]:
cheque = Cheque.write(account=account, recipient_url=access_url, resource=r.fingerprint(), 
                      max_spend=10.0)
print(cheque)

In [ ]:
authorisation = Authorisation(user=user, resource=r.fingerprint())

In [ ]:
args = {"request": r.to_data(),
        "authorisation": authorisation.to_data(),
        "cheque": cheque.to_data()}

In [ ]:
access_service = Service(service_url=access_url)

In [ ]:
result = access_service.call_function(function="run_calculation", args=args)

In [ ]:
user.logout()